In [14]:
import gensim
import pandas as pd
import numpy as np

from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")

In [15]:
model = gensim.models.KeyedVectors.load("araneum_none_fasttextcbow_300_5_2018.model")

In [16]:
data = pd.read_csv('train_1.csv')

In [17]:
with open('forms.csv') as file: # словарь словоформ
    forms = file.readlines()
forms = [word.strip('\"»«\n)(') for word in forms]
forms = [word.lower() for word in forms if word]
labels = pd.Series(0, index=np.arange(len(forms)))
forms = pd.DataFrame({'words': forms, 'values': labels})

In [37]:
df = pd.concat([forms.sample(600), data], axis=0).sample(frac=1)

In [38]:
df['model'] = df.words.apply(lambda x: model[x])

In [49]:
X = np.vstack(df.model.values)
y = df['values'].values

In [50]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [51]:
clf = LogisticRegression().fit(X_train, y_train)

In [52]:
y_pred = clf.predict(X_test)
f1_score(y_test, y_pred)

0.7935368043087971

In [13]:
# for word in words.sample(100).modified.values:
#     res = clf.predict(model[word].reshape(1, -1))
#     if res[0] == 1:
#         print(f"{word}: {res}")

In [37]:
# dataset = pd.read_csv('source_filtered_unique_words.txt', sep='\t', names=['word', 'info', 'modified'])

In [29]:
# ff = open('examples_of_deforming.csv', 'w+')
# 
# for word in dataset.sample(10000).modified.values:
#     res = clf.predict(model[word].reshape(1, -1))
#     if res[0] == 1:
#         example = f"{word}\t{res}" + "\n"
#         ff.write(example)

In [30]:
# examples = pd.read_csv('examples_of_deforming.csv', sep='\t', names=['word', 'class'])
# examples.to_csv('examples_of_deform.csv')
